In [1]:
import numpy as np, pandas as pd, tensorflow as tf
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("iris.csv")

In [5]:
X = df.iloc[:, :-1].values
y = pd.Categorical(df.iloc[:, -1]).codes        # convert class names → integer labels
y = tf.keras.utils.to_categorical(y, 3)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

In [9]:
normalizer = tf.keras.layers.Normalization()
normalizer.adapt(X_train)

In [11]:
def build_mlp(activation, optimizer):
    model = tf.keras.Sequential([
        normalizer,
        tf.keras.layers.Dense(8, activation=activation),
        tf.keras.layers.Dense(3, activation="softmax")
    ])
    model.compile(optimizer=optimizer,
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
    return model

In [13]:
acts = ["relu", "sigmoid", "tanh"]
opts = ["adam", "sgd", "rmsprop"]

results = {}

In [15]:
for a in acts:
    for o in opts:
        model = build_mlp(a, o)
        model.fit(X_train, y_train, epochs=50, batch_size=8,
                  validation_data=(X_test, y_test), verbose=0)
        _, acc = model.evaluate(X_test, y_test, verbose=0)
        results[(a, o)] = acc
        print(f"Activation={a:7} | Optimizer={o:7} | Accuracy={acc:.4f}")

Activation=relu    | Optimizer=adam    | Accuracy=0.8667
Activation=relu    | Optimizer=sgd     | Accuracy=0.8667
Activation=relu    | Optimizer=rmsprop | Accuracy=0.9000
Activation=sigmoid | Optimizer=adam    | Accuracy=0.8333
Activation=sigmoid | Optimizer=sgd     | Accuracy=0.8333
Activation=sigmoid | Optimizer=rmsprop | Accuracy=0.7333
Activation=tanh    | Optimizer=adam    | Accuracy=0.9000
Activation=tanh    | Optimizer=sgd     | Accuracy=0.8000
Activation=tanh    | Optimizer=rmsprop | Accuracy=0.9000


In [16]:
best = max(results.items(), key=lambda x: x[1])
print(f"\nBest → Activation={best[0][0]}, Optimizer={best[0][1]}, Accuracy={best[1]:.4f}")


Best → Activation=relu, Optimizer=rmsprop, Accuracy=0.9000
